In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import load_model

# Load dataset
file_path = '/content/Housing (3).csv'  # Use the correct dataset path
data = pd.read_csv(file_path)

# Identify categorical features
categorical_features = data.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(data[categorical_features])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_features))

# Concatenate encoded features with numerical features
numerical_features = data.drop(columns=categorical_features)
X = pd.concat([numerical_features.drop('price', axis=1), encoded_df], axis=1)
y = data['price']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the model
model = Sequential()
model.add(Dense(15, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(20, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer=Adam(), metrics=['mean_absolute_error'])

# Train the model
history = model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_test, y_test))

# Save the model
model.save('housing_price_model.h5')

# Predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Evaluate performance
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"Training Mean Squared Error: {train_mse}")
print(f"Testing Mean Squared Error: {test_mse}")
print(f"Training Mean Absolute Error: {train_mae}")
print(f"Testing Mean Absolute Error: {test_mae}")

# Load model for prediction
loaded_model = load_model('housing_price_model.h5')

def predict_housing_price(features):
    # Create a DataFrame for user input using numerical and categorical feature column names
    user_input_df = pd.DataFrame([features[:len(numerical_features.columns) - 1]], columns=numerical_features.columns[1:])

    # Extract categorical feature values from user input
    categorical_input_df = pd.DataFrame([features[len(numerical_features.columns) - 1:]], columns=categorical_features)

    # Convert categorical features to string type before transforming
    categorical_input_df = categorical_input_df.astype(str)

    # Apply one-hot encoding to categorical input
    encoded_user_input = encoder.transform(categorical_input_df)
    encoded_user_input_df = pd.DataFrame(encoded_user_input, columns=encoder.get_feature_names_out(categorical_features))

    # Concatenate numerical and encoded categorical inputs
    final_input_df = pd.concat([user_input_df, encoded_user_input_df], axis=1)

    # Standardize the input
    scaled_input = scaler.transform(final_input_df)

    # Predict the housing price
    prediction = loaded_model.predict(scaled_input)
    return prediction[0][0]

# Example input (replace with appropriate values)
user_input = [8000, 3, 2, 2, 1, 1, 'yes', 'no', 'no', 'yes', 'no', 'furnished']  # Adjust according to your data structure
predicted_price = predict_housing_price(user_input)
print(f"Predicted Housing Price: {predicted_price}")


Epoch 1/150


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 25564075261952.0000 - mean_absolute_error: 4704072.0000 - val_loss: 30129973624832.0000 - val_mean_absolute_error: 5007535.0000
Epoch 2/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 24669014982656.0000 - mean_absolute_error: 4656349.0000 - val_loss: 30129948459008.0000 - val_mean_absolute_error: 5007532.5000
Epoch 3/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 25490358272000.0000 - mean_absolute_error: 4715339.5000 - val_loss: 30129885544448.0000 - val_mean_absolute_error: 5007527.0000
Epoch 4/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 25789793828864.0000 - mean_absolute_error: 4742091.0000 - val_loss: 30129732452352.0000 - val_mean_absolute_error: 5007512.5000
Epoch 5/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 24055277158400.0000 - mean_absolute_error: 4612916.5000 - val_loss: 30129378033664.0000 - val_mean_absolute_error: 5007481.0000
Epoch 6/150
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 248451463905

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


Training Mean Squared Error: 837168332800.0
Testing Mean Squared Error: 1909952086016.0
Training Mean Absolute Error: 680254.375
Testing Mean Absolute Error: 1009614.75
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
Predicted Housing Price: 5604796.5
